In [1]:
!pip install transformers accelerate datasets evaluate sentencepiece -q
!pip install evaluate

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from evaluate import load
import torch
import math

In [3]:
!pip install bitsandbytes -q

In [4]:
!pip install -U bitsandbytes -q

In [5]:
model_name = "pierreramez/llama3.1-finetuned-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [6]:
dataset = load_dataset("json", data_files="holdout_evaluation_set.jsonl")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
texts = [x["text"] for x in dataset]

In [8]:
perplexity_metric = load("perplexity")

results = perplexity_metric.compute(
    model_id=model_name,
    predictions=texts,
    batch_size=1
)

  0%|          | 0/1492 [00:00<?, ?it/s]

In [10]:
print(results)

{'perplexities': [3.388671875, 4.484375, 5.8515625, 4.20703125, 3.8671875, 3.796875, 5.5078125, 4.07421875, 2.775390625, 3.337890625, 3.4765625, 5.5, 4.35546875, 5.33203125, 3.859375, 3.470703125, 2.775390625, 2.796875, 3.818359375, 2.736328125, 2.8515625, 3.279296875, 4.9453125, 2.125, 3.623046875, 4.2421875, 3.11328125, 3.05859375, 2.634765625, 4.23046875, 7.66796875, 3.080078125, 3.978515625, 8.1484375, 8.5703125, 5.12890625, 3.146484375, 3.71875, 4.015625, 3.38671875, 4.25390625, 3.318359375, 2.5, 5.66015625, 6.22265625, 5.33203125, 2.60546875, 5.16796875, 4.0234375, 3.458984375, 4.13671875, 3.615234375, 3.1875, 2.68359375, 5.23046875, 3.73046875, 2.50390625, 3.435546875, 3.56640625, 3.927734375, 4.17578125, 3.177734375, 4.2109375, 4.765625, 3.419921875, 9.2109375, 2.951171875, 3.1171875, 6.38671875, 3.939453125, 5.40234375, 3.736328125, 2.916015625, 2.904296875, 9.3984375, 3.978515625, 3.693359375, 3.904296875, 3.28515625, 3.041015625, 2.44140625, 4.96875, 2.546875, 4.20703125, 6.

In [11]:
perps = results["perplexities"]
mean_perplexity = sum(perps) / len(perps)

print("--------------------------------------------------")
print("✅ Perplexity for the fine-tuned model:")
print("➡️ ", mean_perplexity)
print("--------------------------------------------------")

--------------------------------------------------
✅ Perplexity for your fine-tuned model:
➡️  4.088562829884383
--------------------------------------------------


In [19]:
!pip install bert-score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00


In [20]:
import evaluate
bertscore = evaluate.load("bertscore")

In [22]:
print(dataset.column_names)

['messages', 'text']


In [ ]:
references = []
predictions = []

for sample in dataset:
    # Convert messages to string
    prompt = " ".join([msg["content"] for msg in sample["messages"]])
    ref = sample["text"]

    # Generate output
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=150)
    pred = tokenizer.decode(output[0], skip_special_tokens=True)

    references.append(ref)
    predictions.append(pred)

# Compute BERTScore
results = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="en"  # لو عربي خليها "ar"
)

print("Average F1 score:", sum(results["f1"]) / len(results["f1"]))